In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [14]:
# Check entry list for class
# run round counter
# mec round counter
# if mec pull hourly payouts

# check qualifying results
# check race results, update report

# watch live scoring and report houly results, update report


# How to access/display this report???



In [1]:
# 12.20. (SSR) Championship Points Table. Championship points are awarded in each Race to the
# finishers that have met the eligibility requirements in each championship per the following table:



# Driver of the Car during the qualifying segment must start that Car in the Race or its assigned
# grid position and qualifying points are forfeit and the Car moved to the back of the grid in its
# # class

# 40.6.1. (SSR) Qualifying Points are awarded for the results of the qualifying segment (Art. 40.2.),
# per class, at a corresponding rate of 10% of the amounts per position shown in the table listed
# in Art. 12.20. 

In [3]:
#data-str is the points structure as copypasted from the IMSA 2024 regulations 

data_str = """
1st 350 Points 11th 200 Points 21st 100 Points
2nd 320 Points 12th 190 Points 22nd 90 Points
3rd 300 Points 13th 180 Points 23rd 80 Points
4th 280 Points 14th 170 Points 24th 70 Points
5th 260 Points 15th 160 Points 25th 60 Points
6th 250 Points 16th 150 Points 26th 50 Points
7th 240 Points 17th 140 Points 27th 40 Points
8th 230 Points 18th 130 Points 28th 30 Points
9th 220 Points 19th 120 Points 29th 20 Points
10th 210 Points 20th 110 Points 30th+ 10 Points
"""

data_list = []

# Split the data by 'Points' and process each part
for part in data_str.split('Points'):
    # Extract position and points from each part
    elements = part.split()
    if elements:
        position_str, points_str = elements[-2], elements[-1]
        position = int(''.join(char for char in position_str if char.isdigit()))
        points = int(points_str)
        data_list.append((position, points))


data_list_sorted = sorted(data_list, key=lambda x: x[0])

points_tbl = data_list_sorted


In [4]:
points_tbl

[(1, 350),
 (2, 320),
 (3, 300),
 (4, 280),
 (5, 260),
 (6, 250),
 (7, 240),
 (8, 230),
 (9, 220),
 (10, 210),
 (11, 200),
 (12, 190),
 (13, 180),
 (14, 170),
 (15, 160),
 (16, 150),
 (17, 140),
 (18, 130),
 (19, 120),
 (20, 110),
 (21, 100),
 (22, 90),
 (23, 80),
 (24, 70),
 (25, 60),
 (26, 50),
 (27, 40),
 (28, 30),
 (29, 20),
 (30, 10)]

In [5]:
#Importing the complete entry list. Source is Daytona 24hr entrylist as posted on DSC, entry list was not readily available elsewhere

from selenium import webdriver

driver = webdriver.Chrome()
driver.get("https://www.dailysportscar.com/2024/01/02/2024-daytona-24-hours-provisional-entry-list.html")


soup = BeautifulSoup(driver.page_source, 'html.parser')
imsa_entries = soup.find_all('tr')

driver.quit()


In [6]:
imsa_entries

[<tr class="row-1 odd"><th class="column-1 sorting_disabled" colspan="1" rowspan="1" style="width: 33.1188px;">No.</th><th class="column-2 sorting_disabled" colspan="1" rowspan="1" style="width: 70.7875px;">Class</th><th class="column-3 sorting_disabled" colspan="1" rowspan="1" style="width: 186.387px;">Drivers</th><th class="column-4 sorting_disabled" colspan="1" rowspan="1" style="width: 364.019px;">Team</th><th class="column-5 sorting_disabled" colspan="1" rowspan="1" style="width: 266.888px;">Car</th></tr>,
 <tr class="row-2 even">
 <td class="column-1">01</td><td class="column-2">GTP</td><td class="column-3">Sébastien Bourdais<br/>
 Renger van der Zande<br/>
 Scott Dixon</td><td class="column-4">Cadillac Racing</td><td class="column-5">Cadillac V-Series.R</td>
 </tr>,
 <tr class="row-3 odd">
 <td class="column-1">5</td><td class="column-2">GTP</td><td class="column-3">Gianmaria Bruni<br/>
 Neel Jani<br/>
 TBA</td><td class="column-4">Proton Competition</td><td class="column-5">Por

In [28]:
#storing each entries data to lists. Excluding first row as headers from DSC table. New headers will be made in dataframe


car_num = []
car_cls = []
drivers = []
teams = []
vehicle = []


for tr in imsa_entries[1:]:
    car_num.append(tr.find('td', class_='column-1').text)
    car_cls.append(tr.find('td', class_='column-2').text)
    drivers.append(tr.find('td', class_='column-3').text)
    teams.append(tr.find('td', class_='column-4').text)
    vehicle.append(tr.find('td', class_='column-5').text)

In [29]:
drivers

['Sébastien Bourdais\nRenger van der Zande\nScott Dixon',
 'Gianmaria Bruni\nNeel Jani\nTBA',
 'Nick Tandy\nMathieu Jaminet\nKévin Estre\nLaurens Vanthoor',
 'Dane Cameron\nFelipe Nasr\nJosef Newgarden\nMatt Campbell',
 'Ricky Taylor\nFilipe Albuquerque\nBrendon Hartley\nMarcus Ericsson',
 'Jesse Krohn\nAugusto Farfus\nDries Vanthoor\nTBA',
 'Connor de Phillippi\nNick Yelloly\nMaxime Martin\nTBA',
 'Pipo Derani\nJack Aitken\nTom Blomqvist',
 'Jordan Taylor\nLouis Delétraz\nColton Herta\nJenson Button',
 'Richard Westbrook\nPhil Hanson\nBen Keating\nTijmen van der Helm',
 "Ben Keating\nBen Hanley\nNico Pino\nPato O'Ward",
 'George Kurtz\nColin Braun\nToby Sowery\nMalthe Jakobsen',
 'John Farano\nMichael Dinan\nScott McLaughlin\nFerdinand Habsburg',
 'Steven Thomas\nMikkel Jensen\nHunter McElrea\nCharles Milesi',
 'Dwight Merriman\nRyan Dalziel\nConnor Zilisch\nChristian Rasmussen',
 'Dennis Andersen\nSeth Lucas\nLaurents Hörr\nScott Huffaker',
 'Dan Goldburg\nPaul di Resta\nBijoy Garg\n

In [32]:
# Creating the dataframe and splitting the drivers to separate columns

imsa_entries = pd.DataFrame({'car_cls':car_cls, 'car_num':car_num , 'drivers':drivers, 'team':teams, 'vehicle':vehicle})
imsa_entries[['driver_1', 'driver_2', 'driver_3', 'driver_4']] = imsa_entries['drivers'].str.split('\n', expand=True)
imsa_entries = imsa_entries.drop('drivers', axis=1)

In [40]:
imsa_entries

,car_cls,car_num,team,vehicle,driver_1,driver_2,driver_3,driver_4
0,GTP,01,Cadillac Racing,Cadillac V-Series.R,Sébastien Bourdais,Renger van der Zande,Scott Dixon,None
1,GTP,5,Proton Competition,Porsche 963,Gianmaria Bruni,Neel Jani,TBA,None
2,GTP,6,Porsche Penske Motorsport,Porsche 963,Nick Tandy,Mathieu Jaminet,Kévin Estre,Laurens Vanthoor
3,GTP,7,Porsche Penske Motorsport,Porsche 963,Dane Cameron,Felipe Nasr,Josef Newgarden,Matt Campbell
4,GTP,10,Wayne Taylor Racing with Andretti,Acura ARX-06,Ricky Taylor,Filipe Albuquerque,Brendon Hartley,Marcus Ericsson
5,GTP,24,BMW M Team RLL,BMW M Hybrid V8,Jesse Krohn,Augusto Farfus,Dries Vanthoor,TBA
6,GTP,25,BMW M Team RLL,BMW M Hybrid V8,Connor de Phillippi,Nick Yelloly,Maxime Martin,TBA
7,GTP,31,Whelen Cadillac Racing,Cadillac V-Series.R,Pipo Derani,Jack Aitken,Tom Blomqvist,None
8,GTP,40,Wayne Taylor Racing with Andretti,Acura ARX-06,Jordan Taylor,Louis Delétraz,Colton Herta,Jenson Button
9,GTP,85,JDC-Miller MotorSports,Porsche 963,Richard Westbrook,Phil Hanson,Ben Keating,Tijmen van der Helm


In [ ]:
# Capturing the IMSA event schedule information. Storing needed data in a new DataFrame

In [52]:
# url = "https://www.imsa.com/weathertech/weathertech-2024-schedule/"
# response = requests.get(url)
# soup = BeautifulSoup(response.content, "html.parser")
# soup

In [25]:
# pip install selenium

In [53]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.get("https://www.imsa.com/weathertech/weathertech-2024-schedule/")

# Wait for dynamic content to load if needed

soup = BeautifulSoup(driver.page_source, 'html.parser')
schedule_details = soup.find_all('div', {'class': 'schedule-details'})

driver.quit()


In [54]:
schedule_details

[<div class="schedule-details">
 <h2 class="schedule-title" tabindex="0">Roar Before The Rolex 24</h2>
 <div class="schedule-date" tabindex="0">Jan 19 - Jan 21</div>
 <div class="schedule-location" tabindex="0">Daytona International Speedway</div>
 <div class="schedule-duration" tabindex="0">DURATION: N/A</div>
 </div>,
 <div class="schedule-details">
 <h2 class="schedule-title" tabindex="0">Rolex 24 At DAYTONA</h2>
 <div class="schedule-date" tabindex="0">Jan 24 - Jan 28</div>
 <div class="schedule-location" tabindex="0">Daytona International Speedway</div>
 <div class="schedule-duration" tabindex="0">DURATION: 24 Hours</div>
 </div>,
 <div class="schedule-details">
 <h2 class="schedule-title" tabindex="0">Mobil 1 Twelve Hours of Sebring</h2>
 <div class="schedule-date" tabindex="0">Mar 13 - Mar 16</div>
 <div class="schedule-location" tabindex="0">Sebring International Raceway</div>
 <div class="schedule-duration" tabindex="0">DURATION: 12 Hours</div>
 </div>,
 <div class="schedule-d

In [57]:
schedule_details[2]

<div class="schedule-details">
<h2 class="schedule-title" tabindex="0">Mobil 1 Twelve Hours of Sebring</h2>
<div class="schedule-date" tabindex="0">Mar 13 - Mar 16</div>
<div class="schedule-location" tabindex="0">Sebring International Raceway</div>
<div class="schedule-duration" tabindex="0">DURATION: 12 Hours</div>
</div>

In [85]:
event = []
circuit = []
length = []
date = []


for div in schedule_details:
    event.append(div.find('h2', class_='schedule-title').text)
    circuit.append(div.find('div', class_='schedule-location').text)
    length.append(div.find('div', class_='schedule-duration').text)
    date.append(div.find('div', class_='schedule-date').text)
  

In [86]:
event

['Roar Before The Rolex 24',
 'Rolex 24 At DAYTONA',
 'Mobil 1 Twelve Hours of Sebring',
 'Acura Grand Prix of Long Beach',
 'MOTUL Course de Monterey Powered by Hyundai N',
 'Detroit Grand Prix',
 "Sahlen's Six Hours of The Glen",
 'Chevrolet Grand Prix',
 'IMSA SportsCar Weekend',
 'Michelin GT Challenge At VIR',
 'TireRack.com Battle on the Bricks',
 'MOTUL Petit Le Mans']

In [87]:
imsa_sched = pd.DataFrame({'date':date, 'event':event, 'circuit':circuit, 'length':length})
imsa_sched

,date,event,circuit,length
0,Jan 19 - Jan 21,Roar Before The Rolex 24,Daytona International Speedway,DURATION: N/A
1,Jan 24 - Jan 28,Rolex 24 At DAYTONA,Daytona International Speedway,DURATION: 24 Hours
2,Mar 13 - Mar 16,Mobil 1 Twelve Hours of Sebring,Sebring International Raceway,DURATION: 12 Hours
3,Apr 19 - Apr 20,Acura Grand Prix of Long Beach,Long Beach Street Circuit,DURATION: 100 Minutes
4,May 10 - May 12,MOTUL Course de Monterey Powered by Hyundai N,WeatherTech Raceway Laguna Seca,DURATION: 2 Hours 40 Minutes
5,May 31 - Jun 1,Detroit Grand Prix,Detroit Street Circuit,DURATION: 100 Minutes
6,Jun 20 - Jun 23,Sahlen's Six Hours of The Glen,Watkins Glen International,DURATION: 6 Hours
7,Jul 12 - Jul 14,Chevrolet Grand Prix,Canadian Tire Motorsport Park,DURATION: 2 Hours 40 Minutes
8,Aug 2 - Aug 4,IMSA SportsCar Weekend,Road America,DURATION: 2 Hours 40 Minutes
9,Aug 23 - Aug 25,Michelin GT Challenge At VIR,VIRginia International Raceway,DURATION: 2 Hours 40 Minutes


In [3]:
url = "http://results.imsa.com/?season=23_2023&evvent=21_Road+Atlanta"
response = requests.get(url)
results_soup = BeautifulSoup(response.content, "html.parser")
results_soup


<!DOCTYPE html>

<html lang="en">
<head>
<!-- Required meta tags -->
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<!-- Bootstrap CSS -->
<link crossorigin="anonymous" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" rel="stylesheet"/>
<link href="Settings/results.css?v=1704820161.6063" rel="stylesheet"/>
<link href="https://fonts.googleapis.com/css?family=Oswald&amp;display=swap" rel="stylesheet"/>
<link href="https://fonts.googleapis.com" rel="preconnect"/>
<!-- TREEMENU CODE -->
<link href="css/layerstreemenu.css?v=1704820161.6063" rel="stylesheet" type="text/css"/>
<style type="text/css">
				@import url("css/layerstreemenu-hidden.css?v=1704820161.6063");		
		</style>
<link href="favicon.ico" rel="shortcut icon">
<script language="JavaScript" type="text/javascript">

			screen.orientation.lock("portrait